In [1]:
import os
import json
import pandas as pd

In [2]:
def load_json_file(file):
    with open(file) as f:
        data = json.load(f)
    return data['data']

In [3]:
directory = "cro"
file_list = os.listdir(directory)

In [4]:
df = pd.DataFrame()
frames = []
for file in file_list:
    try:
        if file.endswith(".json"):
            filepath = os.path.join(directory, file)
            stanice = file.split("-")[0]
            dataframe = pd.DataFrame(load_json_file(filepath))
            dataframe['stanice'] = stanice
            frames.append(dataframe)
    except:
        print(file)

In [5]:
df = pd.concat(frames)

In [6]:
df = df.drop(columns=['files','itemcode']).sort_values(by='since').reset_index(drop=True)

In [7]:
df

,stanice,since,id,interpret,interpret_id,track,track_id
0,radiowave,2023-10-22T00:00:22+02:00,19376269.0,SYLVAN ESSO,11101.0,Frequency,82943.0
1,radiowave,2023-10-22T00:03:12+02:00,19376274.0,WINONA OAK,25899.0,Winter Rain,87407.0
2,jazz,2023-10-22T00:04:18+02:00,19376275.0,OMRI MOR,15258.0,Tears,51319.0
3,radiowave,2023-10-22T00:07:28+02:00,19376280.0,DIIV,2909.0,Skin Game,69599.0
4,jazz,2023-10-22T00:09:38+02:00,19376283.0,ALFREDO RODRIGUEZ & PEDRITO MARTINEZ,19128.0,Estamos Llegando,63167.0
...,...,...,...,...,...,...,...
633247,radiozurnal,2025-05-05T23:56:06+02:00,22062725.0,ESPEN LIND,3300.0,LUCKY FOR YOU,8453.0
633248,dvojka,2025-05-05T23:56:36+02:00,22062728.0,CHRIS REA,3058.0,JULIA,10065.0
633249,radiowave,2025-05-05T23:57:05+02:00,22062730.0,KAI WHISTON,17961.0,Sweet Distraction,113699.0
633250,jazz,2025-05-05T23:58:35+02:00,22062739.0,AL DI MEOLA,2472.0,God Bird Change,37804.0


In [8]:
df.to_json(os.path.join("data","cro_raw.json"), orient="records")

In [17]:
import polars as pl

In [29]:
kategorie = df['stanice'].drop_duplicates().to_list()
kategorie

['radiowave', 'jazz', 'radiozurnal', 'dvojka', 'pohoda', 'vltava', 'd']

In [33]:
df_pl = pl.from_pandas(df).with_columns(pl.col("since").str.to_datetime()).with_columns(pl.col(["id","track_id","interpret_id"]).cast(int).cast(str)).with_columns(pl.col('stanice').cast(pl.Categorical))

In [37]:
from datetime import date

In [45]:
df_pl.write_parquet(os.path.join("data","cro.parquet"), use_pyarrow=True)

In [47]:
df_pl.shape

(633252, 7)

In [43]:
df_pl.filter(pl.col('since').dt.hour() == 0)

stanice,since,id,interpret,interpret_id,track,track_id
cat,"datetime[μs, UTC]",str,str,str,str,str
"""radiowave""",2023-10-22 00:01:36 UTC,"""19376508""","""NATASHA KMETO""","""9707""","""Nothing Forever""","""80655"""
"""radiowave""",2023-10-22 00:04:36 UTC,"""19376514""","""BONZAI""","""11252""","""Cruel""","""38740"""
"""jazz""",2023-10-22 00:05:08 UTC,"""19376516""","""ROBBEN FORD""","""253""","""Earthquake""","""259"""
"""radiozurnal""",2023-10-22 00:06:13 UTC,"""19376517""","""KEANE""","""2981""","""CRYSTAL BALL""","""16945"""
"""radiowave""",2023-10-22 00:07:59 UTC,"""19376523""","""BARTEES STRANGE""","""24053""","""Mustang""","""81193"""
…,…,…,…,…,…,…
"""pohoda""",2025-05-05 00:56:00 UTC,"""22058207""","""MILAN CHLADIL""","""3562""","""Bylo léto a kvet mák""","""17868"""
"""dvojka""",2025-05-05 00:57:17 UTC,"""22058211""","""GEORGE EZRA""","""8605""","""Budapest""","""25918"""
"""pohoda""",2025-05-05 00:57:21 UTC,"""22058212""","""NAĎA URBÁNKOVÁ""","""3559""","""Tobě náležel čas""","""8963"""
